<h2><center>Diffusion-Correction closure ($D_N$-method)</center></h2>

This notebook summarizes an alternative closure for the spherical harmonics (SH) method, which gives rise to a variation to the $P_N$-approximation, called $D_N$-approximation. This has been presented in [this paper](https://arxiv.org/abs/0907.2099).

This method gives results with higher accuracy than standard $P_N$ at the same truncation level. It specifically helps with in the transition regime (between diffusion and vaccuum) and offers no benefits in the diffusive or transport regime.

The following images compare $P_1$ (left) with $D_1$ (right):

<center>
<img src="spherical_harmonics_method_closures_DN/p1.png" width="350" style="display:inline;margin:1px"><img src="spherical_harmonics_method_closures_DN/d1.png" width="350" style="display:inline;margin:1px">
</center>

However, both methods converge to the same solution for higher order (left:$P_7$ right:$D_7$):

<center>
<img src="spherical_harmonics_method_closures_DN/p7.png" width="350" style="display:inline;margin:1px"><img src="spherical_harmonics_method_closures_DN/d7.png" width="350" style="display:inline;margin:1px">
</center>


The key idea in this paper is to split the radiance field into two parts. One for moments $\le N$ and another part for everything else (the truncation error). Inserting this into the radiative transfer equation(RTE) causes a modified version, which is then expanded twice: First for all moments $\le N$ (using standard SH expansion) and then for all other moments (using a modified expansion operator).

The resulting coupled set of equations still fully model correct light transport without any truncation error, as opposed to the standard expansion approach used with diffusion theories and $P_N$. One set of equations deals with transport of radiation up to moment $N$. These equations are very similar to the standard $P_N$ equations, but have an additional source term which models the redistribution of energy between the $N$ moments and the truncation error. The second set of equation is a single equation which models the transport of the truncation error.

The paper then---under some assumptions and simplifications---derives an approximation to the radiance field for moments $>N$ from this second equation. This approximation is then used in the moment equations and results in a more accurate solution.



### RTE and operator notation

First lets take the time dependent isotropic radiative transfer equation (RTE):

$$
\begin{align}
\frac{\partial L(\vec{x}, \omega) }{\partial t} + (\omega\cdot\nabla)L(\vec{x}, \omega)=-\sigma_t(\vec{x})L(\vec{x}, \omega) + \sigma_s(\vec{x})\int_{\Omega'}f_p(\vec{x}, \omega'\cdot\omega)L(\vec{x}, \omega')\mathbf{d}\omega' + Q(\vec{x}, \omega)
\end{align}
$$

We introduce the following operators to allow for a more concise notation:

$$
\begin{align}
(\mathcal{A}L)(\vec{x}, \omega) &= (\omega\cdot\nabla)L(\vec{x}, \omega)\\
(\mathcal{S}L)(\vec{x}, \omega) &= \sigma_s(\vec{x})\int_{\Omega'}f_p(\vec{x}, \omega'\cdot\omega)L(\vec{x}, \omega')\mathbf{d}\omega'\\
(\mathcal{K}L)(\vec{x}, \omega) &= \sigma_t(\vec{x})L(\vec{x}, \omega) - (\mathcal{S}L)(\vec{x}, \omega)\\
(\mathcal{L}L)(\vec{x}, \omega) &= (\mathcal{A}+\mathcal{K})L(\vec{x}, \omega)
\end{align}
$$

$\mathcal{A}$ is the transport operator and $\mathcal{S}$ the scattering operator. The operator $\mathcal{K}$ models absorption, outscattering and inscattering. The operator $\mathcal{L}$ models all these effects together.

We further define an operator $\mathcal{M}_N$, which models the SH projection:

$$
\begin{align}
(\mathcal{M}_NL)(\vec{x}) &= \int_\Omega L(\vec{x}, \omega)\overline{Y^{lm}}(\omega)\mathbf{d}\omega = L^{lm} \qquad \forall lm \; 0\le l \le N, -l\le m \le l
\end{align}
$$

Note that the number of equations produced by the projection operator $\mathcal{M}_N$, depends on the value of $N$.

The SH reconstruction $L_N(\vec{x}, \omega)$ is defined as:

$$
L_N(\vec{x}, \omega) = \sum_{l=0}^{N}\sum_{m=-l}^{l}L^{lm}(\vec{x})Y^{lm}(\omega)
$$




Using these definitions, we can write the RTE in operator notation:

$$
\partial_t L(\vec{x}, \omega) + \mathcal{L}L(\vec{x}, \omega) = Q(\vec{x}, \omega)
$$


### RTE with deviation term

The core idea behind the $D_N$ method is, to split the radiance field $L$ into two parts. The first represents $L$ in SH of order $N$. The second part represents higher order content in $L$, which can not be captured by SH moments order $N$:

$$
L(\vec{x}, \omega) = L_N(\vec{x}, \omega) + \tilde L_N(\vec{x}, \omega)
$$

The first term on the RHS is the reconstruction of $L$ from its SH coefficients up to order $N$. The term $\tilde L_N$ is called the deviation, and can be understood as the residual between $L$ and its SH approximation of order $N$:

$$
\tilde L_N(\vec{x}, \omega) = L(\vec{x}, \omega) - L_N(\vec{x}, \omega)
$$

Such a deviation can also be defined for the source term $Q$:

$$
Q(\vec{x}, \omega) = Q_N(\vec{x}, \omega) + \tilde Q_N(\vec{x}, \omega)
$$

Now we replace the radiance field $L$ and source $Q$ in the RTE by its SH expansion and deviation. Using the linearity of operator $\mathcal{L}$, we get:

$$
\partial_t \left(L_N\left(\vec{x}, \omega\right)+\tilde L_N\left(\vec{x}, \omega\right)\right) + \mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) = Q_N\left(\vec{x}, \omega\right)+\tilde Q_N\left(\vec{x}, \omega\right)
$$

We now apply the SH projection $\mathcal{M}_N$. Since the deviation is defined as the parts which can not be represented by SH of order $N$, we have the property that $\mathcal{M}_N(\tilde L_N) = 0$ and $\mathcal{M}_N(\tilde Q_N) = 0$. Therefore the moment equations of the RTE are:

$$
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{M}_N\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) = \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)
$$

The third term on the LHS is the deviation term of the RTE. This term has the following intuition behind it: Consider radiance field frequencies for bands higher than $N$. The different interactions $\mathcal{L}$ with the medium (scattering, absorption, transport) will cause energy to be redistributed in terms of frequencies. When we now work with the truncated moment equations, which model radiative transfer for frequency bands up to order $N$, we will have additional energy from the interaction of higher frequencies with the medium being injected into our system for lower frequencies. This injection is modeled by an additional source term: the deviation term. The most popular example for this is optically thick media with a large scattering coefficient. Here the radiance distribution becomes close to isotropic, hence energy is redistributed to lower frequencies.

The deviation term takes the part of the radiance field, which can not be represented by SH coefficients of order $N$, applies the transfer operator $\mathcal{L}$, and projects the result into SH of order $N$. As explained above, this is because $\mathcal{L}$ may produce frequencies $\le N$.





### Standard closure $P_N$

In the SH method, the deviation is assumed to be zero:

$$
\tilde L_N\left(\vec{x}, \omega\right) = 0
$$

This basically models the assumption, that the radiance field can be perfectly represented by a truncated SH expansion without any error. The deviation term in the RTE vanishes and we get the standard $P_N$-equations:

$$
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right) = \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)
$$

### Diffusion-correction closure $D_N$

The key idea behind the $D_N$-approximation is, to derive a better approximation for $\tilde L_N$ and consequently improve the accuracy of the method (while still being an approximation).

We start by introducing the deviation operator $\mathcal{P}_N$:

$$
(\mathcal{P}_NL)(\vec{x}, \omega) = L(\vec{x}, \omega) - L_N(\vec{x}, \omega)
$$

Note that

$$
\begin{align}
(\mathcal{P}_NL_N)(\vec{x}, \omega) &= L_N(\vec{x}, \omega) - \left(\mathcal{M}_N L_N\right)(\vec{x}, \omega) \\
&= L_N(\vec{x}, \omega) - L_N(\vec{x}, \omega)\\
&= 0
\end{align}
$$

and

$$
\begin{align}
(\mathcal{P}_N\tilde L_N)(\vec{x}, \omega) &= \tilde L_N(\vec{x}, \omega) - \left(\mathcal{M}_N \tilde L_N\right)(\vec{x}, \omega) \\
&= \tilde L_N(\vec{x}, \omega) - 0\\
&= \tilde L_N(\vec{x}, \omega)
\end{align}
$$


We now want to find an equation, which models the transfer of our deviation $\tilde L$. This is simply done by taking the RTE (with deviation term) and applying the deviation operator $\mathcal{P}_N$ to each term:

$$
\partial_t \left(\mathcal{P}_N L_N\left(\vec{x}, \omega\right)+\mathcal{P}_N \tilde L_N\left(\vec{x}, \omega\right)\right) + \mathcal{P}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{P}_N\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) = \mathcal{P}_NQ_N\left(\vec{x}, \omega\right)+\mathcal{P}_N\tilde Q_N\left(\vec{x}, \omega\right)
$$

Using the properties of the deviation operator above, this simplifies to a single (six-dimensional) equation:

$$
\partial_t \tilde L_N\left(\vec{x}, \omega\right) + \mathcal{P}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{P}_N\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) = \tilde Q_N\left(\vec{x}, \omega\right)
$$

$\tilde L$ is not zero anymore and therefore the deviation term in the moment equations of the RTE will not vanish. All together we have the following coupled system of equations:

$$
\begin{align}
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{M}_N\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) &= \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)\\
\partial_t \tilde L_N\left(\vec{x}, \omega\right) + \mathcal{P}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{P}_N\mathcal{L}\tilde L_N\left(\vec{x}, \omega\right) &= \tilde Q_N\left(\vec{x}, \omega\right)
\end{align}
$$

The strategy behind $D_N$ now is, to derive an explicit expression for $\tilde L$ from the deviation equation and insert this into the deviation term of the RTE. This is made possible with some key assumptions and simplifications:

- the truncation error does not change with time (dropping the time derivative term in the deviation equation)
- scattering and absorption is invariant under the deviation operator ($\mathcal{P}_N \mathcal{K}=\mathcal{K}$)
- the deviation due to advection is neglectible in comparison to the deviation from scattering and absorption ($\mathcal{P}_N \mathcal{A}=0$)
- finally, the derived expression for $\tilde L_N$ is further being expanded into a Neumann series (using $(Id - T)^{-1}=\sum_k^\infty T^k$) and truncated after the first term

Under these assumptions, the paper derives the following expression for $\tilde L_N$:

$$
\tilde L_N = \mathcal{K}^{-1}(\tilde Q_N - \tilde{\mathcal{P}}_N\mathcal{A}L_N)
$$

this expression is used in the deviation term of the RTE. Applying the spherical harmonics projection $\mathcal{M}_N$ gives:

$$
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{M}_N\mathcal{L}\left(\mathcal{K}^{-1}(\tilde Q_N - \tilde{\mathcal{P}}_N\mathcal{A}L_N)\right)\left(\vec{x}, \omega\right) = \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)
$$

expanding $\mathcal{L}$ in the second term on the RHS gives:

$$
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{M}_N\mathcal{A}\left(\mathcal{K}^{-1}(\tilde Q_N - \tilde{\mathcal{P}}_N\mathcal{A}L_N)\right)\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{K}\left(\mathcal{K}^{-1}(\tilde Q_N - \tilde{\mathcal{P}}_N\mathcal{A}L_N)\right)\left(\vec{x}, \omega\right) = \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)
$$

Since $\mathcal{M}\tilde{Q}_N=0$ and we further assume $\mathcal{P}_N \mathcal{A}=0$, the third term on the LHS vanishes. With this we arrive at the final $D_N$-equations:

$$
\partial_t L_N\left(\vec{x}, \omega\right) + \mathcal{M}_N\mathcal{L}L_N\left(\vec{x}, \omega\right)+\mathcal{M}_N\mathcal{A}\left(\mathcal{K}^{-1}(\tilde Q_N - \tilde{\mathcal{P}}_N\mathcal{A}L_N)\right)\left(\vec{x}, \omega\right) = \mathcal{M}_NQ_N\left(\vec{x}, \omega\right)
$$

the deviation term contains two advection operators and therefore involves two spatial derivatives ($\nabla$). This is why the term is said to be of diffusive nature and hence gives the closure its name. After expanding the terms, one further sees that this term vanishes in all moments $<N$ and only appears in the equation for the last moment $N$.



### Relationship to the closure problem in diffusion approximation based methods

It is interesting to relate the diffusion-correction closure to the closure problem of the diffusion approximation and variable eddington factor(VEF) approximations (see report version 5, chapter 2.2 on $P_1$ moment closure).

In the report, the moment problem is defined as finding an expression for the $N+1$-moment of the radiance field distribution (in angular domain). This is because the moment expansion equations truncated at order $N$ depend only on the moments up to order $N+1$. This can best be seen in the *matrix M section* in the notebook starmap_extensions_arte_study.ipynb. The directional derivative term causes a block-tridiagonal structure with an empty main block-diagonal and it can be seen that the coefficients of band $l$ depend only on coefficients up to band $l+1$. It is important to note, that this dependency is broken, once we have an anisotropic medium.

For different diffusion closures (such as VEF), an assumption on the radiance field distribution was made, of which its moment $N+1$ (under further assumptions and simplications) was expressed in terms of the moments $< N$. Inserting this derived moment led to non-linear diffusions coefficients and a variety of diffusion theories. The closure problem for diffusion theories therefore was defined as finding an expression for the $N+1$-th moment of the radiance field distribution in terms of the $N$ moments.

With the diffusion-correction discussed in this notebook, the closure problem is defined from a different perspective. Instead of defining a missing moment in terms of known moments, based on assumptions about the local radiance field distribution, the RTE is extended with a deviation term, which models the interaction of the truncation error with the medium. A coupled set of equations is being derived which still fully models correct light transport, without any truncation error.

The difference between diffusion closures and the closure derived here is, that for the former, it was all about constructing a moment for the next higher band above the truncation level, while the latter is about finding a good approximation for all higher bands using the deviation equation.

However, since for isotropic media, truncated moment equations only depend on radiance field of order $N+1$, both closure perspectives end up doing a very similar thing. In the report we have seen that VEF-closures depend on aggressive simplifications and assumptions and we see something similar in the derivation of the expression for $\tilde L$.

### Relationship to $FP_N$

It is also interesting to note, that the $FP_N$ method also contains an additional source term in the RTE. Therefore a theoretical connection may be established. While the source term in $FP_N$ has been derived from filtering the radiance field, the term in $D_N$ has been derived from correcting the truncation error. In both cases the additional term serves as a way to correct the solution.